# <font style="color:blue">Introduction to Densepose</font>

Earlier, we have learnt multiple networks for object classification, localization and mask generation. Similar to Mask RCNN, an extension of Faster RCNN which adds a third branch to the network for the mask output, lets see how it is further extended to generate 3D surface based model of the human body.

Research in human understanding aims primarily at localizing a sparse set of joints, like the wrists, or elbows of humans. This may suffice for applications like gesture or action recognition, but it delivers a reduced image interpretation. If we go further like trying on new clothes via a photo or putting costumes on your friend’s photos. For these types of tasks, a more complete, surface-based image interpretation is required.

In this module, we will learn how facebook research Densepose achieves dense human pose estimation, which maps all human pixels of 2D RGB images to a 3D surface based model.

<img src='https://www.learnopencv.com/wp-content/uploads/2020/05/c3-w13-densepose_rcnn.png' width=900>



## <font style="color:green">1. Model Architecture</font>

DensePose RCNN introduces improved architectures by combining the DenseReg approach with the Mask-RCNN architecture.

### <font style="color:green">1.1. Fully convolutional Densepose Regression</font>
- A fully convolutional network (FCN) that combines a classification and a regression task, similar to DenseReg. It performs following 2 steps:

**Classification** - Classifies each pixel to which part(I) it belongs.

**Regression** - Regresses each pixel to the corresponding 2D parameterization(U, V) to the part it belongs to

### <font style="color:green">1.2. Region-based Dense Pose Regression</font>

- Using FCN makes the training easy, but to load the same deep network with too many tasks, including part segmentation and pixel localization along with scale invariance becomes challenging for humans in COCO. So, they have adopted the region based approach of Mask RCNN which uses cascade of regions of interest extracting region adapted features through ROI pooling and feeds to the region specific branch.

- Hence, Mask-RCNN with the Feature Pyramid Network (FPN) features, and ROI-Align pooling provides dense part labels and coordinates within each of the selected regions. 

- On top of it, a fully convolutional network is introduced, similar to DenseReg, which combines classification and regression task as shown in the figure below. In this way, it gives per pixel IUV mapping.

I - Patch Index
U, V - 2D parameters of 3D surface

<img src="https://www.learnopencv.com/wp-content/uploads/2020/05/c3-w13-densepose_arch.png" width=900>

### <font style="color:green">1.3. Multi-task cascaded architectures</font>
- Cascading can improve performance both by providing context to the following stages, and also through the benefits of deep supervision. Here, they have also performed **Cross-cascading** architecture: The output of the
RoIAlign module in the above figure feeds into the DensePose network as well as auxiliary networks for other tasks (masks, keypoints). Once first-stage predictions are obtained from all tasks, they are combined and then fed into a second-stage refinement unit of each branch.

<img src='https://www.learnopencv.com/wp-content/uploads/2020/05/c3-w13-densepose_arch1.png' width=900>

Now, lets look into DenseReg system how it establishes dense correspondences between 3D object model and an image in the wild.

## <font style="color:green">2. Dense correspondence - (2D to 3D)</font>

Each 3D object can be defined using 2D parameters UV coordinates. UV mapping is the unwrapped version of the 3D model. In order to fit a 3D template, the dense correspondence is established from the image to the 2D U-V parameterization, which can be used to identify any node on the surface.

We can visualize the bijective mapping function applied to 3D surface coordinate to 2D space as the cylindrical unwrapping. From this cylindrical unwrapping, we can interpret these coordinates as being the horizontal and vertical coordinates while moving on the surface, followed by normalization: u ∈ [0, 1] and v ∈ [0, 1].

<img src='https://www.learnopencv.com/wp-content/uploads/2020/05/c3-w13-UV_mapping.png' width=700>

The above image is from DenseReg paper where the mapping is used only for facial part. In DensePose, we apply same method for Head, Torso, Lower/Upper Arms, Lower/Upper Legs, Hands and Feet.

## <font style="color:green">3. Dataset</font>

**DensePose-COCO** is a large-scale ground-truth dataset with image-to-surface correspondences manually annotated on 50K COCO images. It introduces the first manually-collected ground truth dataset for the task, by gathering dense correspondences between the SMPL model and persons appearing in the COCO dataset. This is accomplished through a novel annotation pipeline that exploits 3D surface information during annotation. It also uses few datasets where images are annotated with keypoints and semi-automated fits of 3D models to images.

<img src="https://www.learnopencv.com/wp-content/uploads/2020/05/c3-w13-densepose_dataset.png" width=900>

Human annotations are done for establishing dense correspondences between 3D surface to 2D image. In order to handle the complex geometry of the human shape, they manually partition the surface into 25 semantic regions (24 body parts + 1 for background). Then each region is sampled with a set of roughly equidistant points and maximum points per part is 14. Each part is parameterized to U(middle), V(right) coordinates as shown above.

<img src="https://www.learnopencv.com/wp-content/uploads/2020/05/c3-w13-partition.png" width=300>

Typically in pose estimation annotations, one asks multiple annotators to label the same landmark, which is then used to assess the variance in position. Here for human annotations accuracy, since they have access to the true mesh coordinates of some images, they have directly compared the ground truth position and the one estimated by annotators. They use the geodesic distance between the correct surface point, and the point estimated by human annotators.

<img src='https://www.learnopencv.com/wp-content/uploads/2020/05/c3-w13-geodesic.png'>
where g(·, ·) measures the geodesic distance between two surface points. The <a href="https://arxiv.org/pdf/1802.00434.pdf">paper</a> also gives the error distribution for different body parts and explains annotator errors are smaller on small surface parts like face, hands, feet when compared to larger uniform areas that are typically covered by clothes like torso, back, hips.

Densepose also adopts a learning based approach for annotation where it trains a teacher network to reconstruct the ground truth values wherever these are observed and deploys on the full image domain, yielding a dense supervision signal. In this way, it is observed to perform better by inpainting the values of supervision signal where they are not annotated.

<img src="https://www.learnopencv.com/wp-content/uploads/2020/05/c3-w13-teacher_net.png" width=900>

## <font style="color:green">4. Different Visualization formats</font>

Densepose annotation has the same format as COCO.

```
{
  "images" : [image],
  "annotations" : [annotation],
  "categories" : [category]
}
```

Sample annotation of Densepose looks like below:
```
annotation {
  "area": float,
  "bbox": [x, y, width, height],
  "category_id": int,
  "dp_I": [float],
  "dp_U": [float],
  "dp_V": [float],
  "dp_masks": [dp_mask],
  "dp_x": [float],
  "dp_y": [float],
  "id": int,
  "image_id": int,
  "iscrowd": 0 or 1,
  "keypoints": [float],
  "segmentation": RLE or [polygon]
}
```

- dp_masks - All part masks are of size 256x256 and maps to 14 labels.  They correspond to 14 semantically meaningful parts of the body: `Torso, Right Hand, Left Hand, Left Foot, Right Foot, Upper Leg Right, Upper Leg Left, Lower Leg Right, Lower Leg Left, Upper Arm Left, Upper Arm Right, Lower Arm Left, Lower Arm Right, Head`. Please note that these are not linked to the 3D template and used to sample annotation points.

- dp_x, dp_y: spatial coordinates of collected points on the image. The coordinates are scaled such that the bounding box size is 256x256.

- dp_I: The patch index that indicates which of the 24 surface patches the point is on. Patches correspond to the body parts described above. Some body parts are split into 2 patches: `1, 2 = Torso, 3 = Right Hand, 4 = Left Hand, 5 = Left Foot, 6 = Right Foot, 7, 9 = Upper Leg Right, 8, 10 = Upper Leg Left, 11, 13 = Lower Leg Right, 12, 14 = Lower Leg Left, 15, 17 = Upper Arm Left, 16, 18 = Upper Arm Right, 19, 21 = Lower Arm Left, 20, 22 = Lower Arm Right, 23, 24 = Head`.

- dp_U, dp_V: Coordinates in the UV space. Each surface patch has a separate 2D parameterization.

**Detectron2** provides densepose models and code for inference and training.
Densepose annotation data contains S, X, Y, I, U, V parameters and the output contains S, I, U, V  parameters. 
The different visualizations provided by detectron2's densepose are:
- **bbox** - bounding boxes of detected persons;
- **dp_segm** - segmentation masks for detected persons;
- **dp_u** - each body part is colored according to the estimated values of the U coordinate in part parameterization;
- **dp_v** - each body part is colored according to the estimated values of the V coordinate in part parameterization;
- **dp_contour** - plots contours with color-coded U and V coordinates

All these methods extract the data from densepose's output IUV mapping  to give different visualizations.

## <font style="color:green">5. Evaluation for corresponse matching</font>

Densepose has introduced geodesic point similarity (GPS) as a correspondence matching score, inspired from object keypoint similarity(OKS) measure.

<img src='https://www.learnopencv.com/wp-content/uploads/2020/05/c3-w13-GPS.png'>

where $P_j$ is the set of ground truth points annotated on person instance $j$, $i_p$ is the vertex estimated by a model at point $p$, $i_p$ is the ground truth vertex $p$ and $κ$ is a normalizing parameter. They set $κ=0.255$ so that a single point has a $GPS$ value of $0.5$ if its geodesic distance from the ground truth equals the average half-size of a body segment. Intuitively, this means that a score of $GPS ≈ 0.5$ can be achieved by a perfect part segmentation model.

## <font style="color:green">References</font>

- https://arxiv.org/pdf/1802.00434.pdf - Densepose Paper
- https://arxiv.org/pdf/1803.02188.pdf - DenseReg Paper
- https://github.com/facebookresearch/detectron2/tree/master/projects/DensePose
- https://github.com/facebookresearch/DensePose
- https://github.com/facebookresearch/DensePose/blob/master/challenge/2019_COCO_DensePose/data_format.md